###Board Game Geek - Spark ETL Project
Board Game Geek is an online site that provides rankings for various boardgames.  A complete listing of ranked games is provided here: https://boardgamegeek.com/browse/boardgame

The project is to scrape board game ranking data from BoardGameGeek and store it as a partitioned Parquet table for analytics.


The fields in the final table are:

* Rank - the integer value from the Board Game Rank column
* ImageURL - the URL (as provided) of the image presented in the second column 
* Title - the title of the game
* YearPublished - the year the game was published
* GeekRating - the float value presented in the Geek Rating field
* AvgRating - the float value presented in the Avg Rating field
* NumVoters - the integer value presented in the NumVoters field

In [0]:
# run this code to make sure beautiful soup is installed
# dbutils.library.installPyPI('beautifulsoup4')
%pip install beautifulsoup4
dbutils.library.restartPython()

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# retrieve the web page

from urllib import request as url

request = url.Request('https://boardgamegeek.com/browse/boardgame')
response = url.urlopen(request)

html = response.read()

print(html)

b'<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tvar width = document.documentElement.clientWidth || window.innerWidth;\n\t\t\t\tif (width < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\t// android debugging\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://

In [0]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
print(soup)

<!DOCTYPE html>

<html lang="en-US" ng-app="GeekApp" ng-cloak="">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" id="vp" name="viewport"/>
<script>
			window.addEventListener( 'DOMContentLoaded',  function() {
				var width = document.documentElement.clientWidth || window.innerWidth;
				if (width < 960) {
					var mvp = document.getElementById('vp');
					// android debugging
					mvp.setAttribute('content','width=960');
				}
			});
		</script>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Browse Board Games | BoardGameGeek</title>
<link href="https://cf.geekdo-static.com/icons/touch-icon180.png" rel="apple-touch-icon"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="icon" type="image/ico"/>
<link href="/game-opensearch.xml" rel="search" title="BGG

In [0]:
# extract the data from the web page   

import pandas as pd

soup = BeautifulSoup(html, 'html.parser')

all_data = []

# get all tables in the page
tables = soup.find_all('table')

# Extract header
header = []
for th in tables[0].find_all('th'):
  header.append(th.get_text(strip=True))

# get data from each table
for table in tables:
  table_data = []
  for tr in table.find_all('tr'):
      row_data = []
      for td in tr.find_all('td'):
          img_tag = td.find('img')
          if img_tag:
              row_data.append(img_tag['src'])
          else:
              row_data.append(td.get_text(strip=True))
      table_data.append(row_data)
  all_data.append(table_data)
  
print(header)
print(all_data)

['Board Game Rank', 'Thumbnail image', 'Title', 'Geek Rating', 'Avg Rating', 'Num Voters', 'Shop']
[[[], ['1', 'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg', 'Brass: Birmingham(2018)Build networks, grow industries, and navigate the world of the Industrial Revolution.', '8.417', '8.60', '44343', ''], ['2', 'https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png', 'Pandemic Legacy: Season 1(2015)Mutating diseases are spreading around the world - can your team save humanity?', '8.383', '8.53', '52964', ''], ['3', 'https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg', 'Gloomhaven(2017)Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!', '8.361', '8.60', '61512', ''], ['4', 'https://cf.geekdo-i

In [0]:
# convert the data to a Spark SQL table
df = pd.DataFrame(all_data[0], columns=header)

BGG = spark.createDataFrame(df)

display(BGG)

Board Game Rank,Thumbnail image,Title,Geek Rating,Avg Rating,Num Voters,Shop
null,null,null,null,null,null,null
1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,"Brass: Birmingham(2018)Build networks, grow industries, and navigate the world of the Industrial Revolution.",8.417,8.60,44343,
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1(2015)Mutating diseases are spreading around the world - can your team save humanity?,8.383,8.53,52964,
3,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven(2017)Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!,8.361,8.60,61512,
4,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM4N8pQ__micro/img/LSAi1pmhbTbWwtBrziutDOXzfdY=/fit-in/64x64/filters:strip_icc()/pic6293412.jpg,"Ark Nova(2021)Plan and build a modern, scientifically managed zoo to support conservation projects.",8.333,8.54,41305,
5,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,"Twilight Imperium: Fourth Edition(2017)Build an intergalactic empire through trade, research, conquest and grand politics.",8.243,8.60,23240,
6,https://cf.geekdo-images.com/PhjygpWSo-0labGrPBMyyg__micro/img/mfU7-dEFnVyXJrF-BWAO0buexao=/fit-in/64x64/filters:strip_icc()/pic5666597.jpg,"Dune: Imperium(2020)Influence, intrigue, and combat in the universe of Dune.",8.225,8.43,43405,
7,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars(2016)Compete with rival CEOs to make Mars habitable and build your corporate empire.,8.216,8.36,97186,
8,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition(2011)The Fellowship and the Free Peoples clash with Sauron over the fate of Middle-earth.,8.180,8.54,20909,
9,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion(2020)Vanquish monsters with strategic cardplay in a 25-scenario Gloomhaven campaign.,8.174,8.45,33323,


In [0]:
# cleanup
import pyspark.sql.functions as f
from datetime import datetime
import re

# extract year
def extract_year_from_title(title):
    year_match = re.search(r'\((\d{4})\)', title)
    if year_match:
        return int(year_match.group(1))
    else:
        return None
      
extract_year_udf = f.udf(extract_year_from_title)

# clean up data in the dataframe
clean_BGG = (
  BGG
    .distinct() # remove duplicate rows
    .withColumnRenamed("Board Game Rank", "Rank")
    .withColumnRenamed("Thumbnail image", "ImageURL")
    .withColumnRenamed("Geek Rating", "GeekRating")
    .withColumnRenamed("Avg Rating", "AvgRating")
    .withColumnRenamed("Num Voters", "NumVoters")
    .drop("shop")
    .dropna(how='any')
    .withColumn('YearPublished', extract_year_udf(f.col('Title')))
    .withColumn('currentdate', f.lit(datetime.now().strftime("%Y-%m-%d")))
)

clean_BGG = clean_BGG.withColumn('Title', f.substring_index(clean_BGG['Title'], '(', 1))

clean_BGG = clean_BGG.select('Rank', 'ImageURL', 'Title', 'YearPublished', 'GeekRating', 'AvgRating', 'NumVoters', 'currentdate')
    
display(clean_BGG)

Rank,ImageURL,Title,YearPublished,GeekRating,AvgRating,NumVoters,currentdate
5,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,Twilight Imperium: Fourth Edition,2017,8.243,8.60,23240,2024-03-30
8,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition,2011,8.180,8.54,20909,2024-03-30
4,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM4N8pQ__micro/img/LSAi1pmhbTbWwtBrziutDOXzfdY=/fit-in/64x64/filters:strip_icc()/pic6293412.jpg,Ark Nova,2021,8.333,8.54,41305,2024-03-30
11,https://cf.geekdo-images.com/kjCm4ZvPjIZxS-mYgSPy1g__micro/img/sRPer5FmBxyV527MY67P-gM7ukg=/fit-in/64x64/filters:strip_icc()/pic7013651.jpg,Spirit Island,2017,8.146,8.35,50055,2024-03-30
3,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven,2017,8.361,8.60,61512,2024-03-30
7,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars,2016,8.216,8.36,97186,2024-03-30
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1,2015,8.383,8.53,52964,2024-03-30
12,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,2017,8.120,8.39,26609,2024-03-30
6,https://cf.geekdo-images.com/PhjygpWSo-0labGrPBMyyg__micro/img/mfU7-dEFnVyXJrF-BWAO0buexao=/fit-in/64x64/filters:strip_icc()/pic5666597.jpg,Dune: Imperium,2020,8.225,8.43,43405,2024-03-30
9,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion,2020,8.174,8.45,33323,2024-03-30


In [0]:
# convert the data to a Spark SQL table
spark.sql('CREATE DATABASE IF NOT EXISTS games')
 
# save ufo sightings data as a table
(
  clean_BGG
    .write
    .partitionBy('currentdate')
    .format('parquet')
    .mode('overwrite')
    .option('overwriteSchema','true')
    .option('path','/tmp/games')
    .saveAsTable('games.board_games')
  )
 
# read data from table
display(
  spark.table('games.board_games')
  )

Rank,ImageURL,Title,YearPublished,GeekRating,AvgRating,NumVoters,currentdate
5,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,Twilight Imperium: Fourth Edition,2017,8.243,8.60,23240,2024-03-30
8,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition,2011,8.180,8.54,20909,2024-03-30
4,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM4N8pQ__micro/img/LSAi1pmhbTbWwtBrziutDOXzfdY=/fit-in/64x64/filters:strip_icc()/pic6293412.jpg,Ark Nova,2021,8.333,8.54,41305,2024-03-30
11,https://cf.geekdo-images.com/kjCm4ZvPjIZxS-mYgSPy1g__micro/img/sRPer5FmBxyV527MY67P-gM7ukg=/fit-in/64x64/filters:strip_icc()/pic7013651.jpg,Spirit Island,2017,8.146,8.35,50055,2024-03-30
3,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven,2017,8.361,8.60,61512,2024-03-30
7,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars,2016,8.216,8.36,97186,2024-03-30
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1,2015,8.383,8.53,52964,2024-03-30
12,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,2017,8.120,8.39,26609,2024-03-30
6,https://cf.geekdo-images.com/PhjygpWSo-0labGrPBMyyg__micro/img/mfU7-dEFnVyXJrF-BWAO0buexao=/fit-in/64x64/filters:strip_icc()/pic5666597.jpg,Dune: Imperium,2020,8.225,8.43,43405,2024-03-30
9,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion,2020,8.174,8.45,33323,2024-03-30


Execute the following SQL statements to validate your results:

In [0]:
%sql -- verify row count

SELECT COUNT(*) FROM games.board_games;

count(1)
100


In [0]:
%sql -- verify values

SELECT * FROM games.board_games ORDER BY rank ASC;

Rank,ImageURL,Title,YearPublished,GeekRating,AvgRating,NumVoters,currentdate
1,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,Brass: Birmingham,2018,8.417,8.60,44343,2024-03-30
10,https://cf.geekdo-images.com/7SrPNGBKg9IIsP4UQpOi8g__micro/img/nEvTiCkWpT-ymH4bstc9c335TtQ=/fit-in/64x64/filters:strip_icc()/pic4325841.jpg,Star Wars: Rebellion,2016,8.171,8.42,31966,2024-03-30
100,https://cf.geekdo-images.com/36WIe0ZHkp5OvHOlB-8vog__micro/img/7lY7AE0KsFAO2svml1DMpa20-kg=/fit-in/64x64/filters:strip_icc()/pic4356580.jpg,Concordia Venus,2018,7.530,8.29,5587,2024-03-30
11,https://cf.geekdo-images.com/kjCm4ZvPjIZxS-mYgSPy1g__micro/img/sRPer5FmBxyV527MY67P-gM7ukg=/fit-in/64x64/filters:strip_icc()/pic7013651.jpg,Spirit Island,2017,8.146,8.35,50055,2024-03-30
12,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,2017,8.120,8.39,26609,2024-03-30
13,https://cf.geekdo-images.com/pNCiUUphnoeWOYfsWq0kng__micro/img/ZlfJiIATIt5MY7xWSMzFmdygaa0=/fit-in/64x64/filters:strip_icc()/pic3530661.jpg,Twilight Struggle,2005,8.070,8.25,48370,2024-03-30
14,https://cf.geekdo-images.com/fVwPntkJKgaEo0rIC0RwpA__micro/img/77dZJpqtydc8TTecl0ReGF7iAVw=/fit-in/64x64/filters:strip_icc()/pic2663291.jpg,Through the Ages: A New Story of Civilization,2015,8.064,8.30,31249,2024-03-30
15,https://cf.geekdo-images.com/u1l0gH7sb_vnvDvoO_QHqA__micro/img/xeShpQfbj4L1id-ItJbfIYVzgyA=/fit-in/64x64/filters:strip_icc()/pic4887376.jpg,Great Western Trail,2016,8.035,8.20,40417,2024-03-30
16,https://cf.geekdo-images.com/5CFwjd8zTcGYVUnkXh04hw__micro/img/N1VrEUaW6bmwnqbZfi2nRScwGxM=/fit-in/64x64/filters:strip_icc()/pic1176894.jpg,The Castles of Burgundy,2011,8.013,8.14,61348,2024-03-30
17,https://cf.geekdo-images.com/7k_nOxpO9OGIjhLq2BUZdA__micro/img/ETplc22GdhlN5u0mppcDxSP8LVk=/fit-in/64x64/filters:strip_icc()/pic3163924.jpg,Scythe,2016,7.997,8.15,82141,2024-03-30


In [0]:
%sql -- verify partitioning
DESCRIBE EXTENDED games.board_games;

col_name,data_type,comment
Rank,string,null
ImageURL,string,null
Title,string,null
YearPublished,string,null
GeekRating,string,null
AvgRating,string,null
NumVoters,string,null
currentdate,string,null
# Partition Information,,
# col_name,data_type,comment
